# Scratch and old functions

In [ ]:
#isochronous transform for one point i
def isochronous_transform_process(t_serie, v_serie, dt, i) :
    t_i  = i*dt
    m  = 0
    t1 = -1
    while(t1 <= t_i) :
        m += 1;
        t1 = t_serie[m]
    v1 = v_serie[m]
    t0 = t_serie[m-1]
    v0 = v_serie[m-1]
    v_i  = lin_interp(v1,t1,v0,t0,t_i)
    return v_i
#parallel isochronous transform over iso time ti
def parallel_isochronous_transform(t_serie, v_serie, dt) :
    T_end    = t_serie[-1]
    Nt_iso   = int(T_end/dt)-1
    v_iso    = Parallel(n_jobs=-2)(
        delayed(isochronous_transform_process)
        (t_serie, v_serie, dt, i) for i in range(Nt_iso))
    return v_iso

In [ ]:
#Dictionnary to store isochronous transforms with various dt
v_iso  = {}; vname = DIR+'Processed_data/v_iso.dat';
t_iso  = {} #time (linspace(0,T,dt))
Nt_iso = {} #number of points
dt_iso = {}; dtname = DIR+'Processed_data/dt_iso.dat';
RERUN  = True
if  (os.path.isfile(vname) and os.path.isfile(dtname)) and (not RERUN) :
    print('Load v_iso.dat...')
    v_iso = Load_data(vname)
    print('Load dt_iso.dat...')
    dt_iso = Load_data(dtname)
    print('Done.')
RESAVE = False
for key in vDNS :
    if key not in v_iso :
        N_ = int(0.001*len(tDNS[key]))
        print("missing " + key + "data..")
        RESAVE = True
        dt = dtDNS_mean[key]
        start = time.time()
        v_iso [key]  = parallel_isochronous_transform(tDNS_ssl[key][0:N_], vDNS_ssl[key][0:N_], dt)
        Nt_iso[key]  = len(v_iso[key])
        t_iso [key]  = np.linspace(0,dt*Nt_iso[key],Nt_iso[key])
        dt_iso[key]  = [dt]
        end = time.time()
        print('\tdt = '+str(dt))
        print('\t Nt_iso = '+str(Nt_iso[key]))
        print('\tTime needed \t: '+str(end-start))
if RESAVE :
    print('Saving data...')
    Save_data(v_iso,vname)
    Save_data(dt_iso,dtname)
    print('Done.')
else : print('All data seem to be loaded.')

In [ ]:
DATA = {}

#Results with CSF = 1
DM_1  = ['0.0E0']
KEYS_1 = ['Bent 1000, D='+dm+', CSF=1' for dm in DM_1]
for dm in DM_1 : DATA['Bent 1000, D='+dm+', CSF=1'] = DIR+CASE+'/sl_out_M0_DM'+dm+'_SLSM20000_TMAX0_CSF1.dat'
PLOT_KEYS_1 = ['Bent 1000, D='+dm_+', CSF=1' for dm_ in DM_1]

#Results with CSF = 40
DM_40 = ['0.0E0',
       '1.0E-13',
       '2.0E-13', 
       '5.0E-13', 
       '1.0E-12',
       '2.0E-12', 
       '5.0E-12', 
       '1.0E-11', 
       '2.0E-11', 
       '5.0E-11', 
       '1.0E-10',
       '2.0E-10',
       '5.0E-10',
       '1.0E-9',
       '1.0E-8',
       '1.0E-7',
       '1.0E-6',
       '1.0E-5']
KEYS_40   = ['Bent 1000, D='+dm+', CSF=40' for dm in DM_40] #ORDERED ARRAY OF KEYS (IMPORTANT TO SORT DATA)
for dm in DM_40 : DATA['Bent 1000, D='+dm+', CSF=40'] = DIR+CASE+'/sl_out_M0_DM'+dm+'_SLSM0_TMAX2.0E7_CSF40.dat'
PLOT_KEYS_40 = ['Bent 1000, D='+dm_+', CSF=40' for dm_ in ['0.0E0','1.0E-13','1.0E-11','1.0E-9','1.0E-8','1.0E-7','1.0E-6','1.0E-5']]

#Results with CSF = 40
DM_300 = ['0.0E0',
       '1.0E-13',
       '1.0E-12',
       '1.0E-11', 
       '1.0E-10',
       '1.0E-9',
       '1.0E-8']
KEYS_300 = ['Bent 1000, D='+dm+', CSF=300' for dm in DM_300] #ORDERED ARRAY OF KEYS (IMPORTANT TO SORT DATA)
for dm in DM_300 : DATA['Bent 1000, D='+dm+', CSF=300'] = DIR+CASE+'/sl_out_M0_DM'+dm+'_SLSM0_TMAX1.0E8_CSF300.dat'
PLOT_KEYS_300 = ['Bent 1000, D='+dm_+', CSF=300' for dm_ in DM_300]

color_cycle = plt.rcParams['axes.prop_cycle'].by_key()['color']
print('Dataset to load : ' +str(len(DATA)))

KEYS = KEYS_1 + KEYS_40 + KEYS_300
PARAM = {}
param = ['DM','SLSM','TMAX','CSF']
for key in DATA : PARAM[key] = Extract_param (DATA[key], param)
D_VALUES_1   = [PARAM[key]['DM'] for key in KEYS_1]
D_VALUES_40  = [PARAM[key]['DM'] for key in KEYS_40]
D_VALUES_300 = [PARAM[key]['DM'] for key in KEYS_300]    
D_VALUES     = D_VALUES_1 + D_VALUES_40 + D_VALUES_300

CSF_VALUES = ['1','40','300']
KEYS_CSF = {'1' : KEYS_1, '40' : KEYS_40, '300' : KEYS_300}

for key in DATA :
    if not os.path.isfile(DATA[key]) :
        print('Warning file from key : ' + key + ' :')
        print('\t' + DATA[key])
        print('not found')

##### Theta isochronous transform

In [ ]:
'''
thetaname = DIR+'Processed_data/theta_iso.dat'
if  os.path.isfile(thetaname) and (not RERUN):
    print('Loading theta_iso.dat...')
    theta_iso = Load_data(thetaname)

RESAVE_THETA = False
iso_missing = {}
MISSING_KEYS = []
for key in KEYS : 
    if key not in theta_iso : 
        RESAVE_THETA = True
        print('Missing '+key+' in theta_iso..')
        MISSING_KEYS.append(key)

if MISSING_KEYS :
    start = time.time()
    iso_missing  = parallel_isochronous_transform(tDNS, thetaDNS, dtDNS_mean, Npoints, dt_factor, MISSING_KEYS)
    for key in iso_missing :
        theta_iso[key] = iso_missing[key]
    end = time.time()
    print('\tTime needed \t: '+str(end-start)) 
iso_missing = {}

if RESAVE_THETA :
    print('Saving '+thetaname+'...')
    tosave = {k_ : theta_iso[k_] for k_ in KEYS}
    Save_data(tosave,thetaname)
tosave = {}
'''